In [ ]:
!pip install transformers
!pip install sentencepiece

Upload model(model.zip)

In [ ]:
from google.colab import files
uploaded = files.upload()

Unzip

In [ ]:
!unzip model.zip

In [ ]:
from transformers import XLNetTokenizer, XLNetForSequenceClassification
from transformers import BertTokenizerFast, BertForSequenceClassification
import torch
device='cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = XLNetTokenizer.from_pretrained('./model')
# model = BertForSequenceClassification.from_pretrained('./model',problem_type="multi_label_classification")
model = XLNetForSequenceClassification.from_pretrained('./model')
model.to(device)

Prepare predict function

In [ ]:
model.eval()
import torch.nn.functional as F
#this will predict one sentence each time
def predict(content):

    inputs = tokenizer(content,
                       
                       padding='max_length',
                       truncation=True, return_tensors="pt")
    # move to gpu
    ids = inputs["input_ids"].to(device)
    idt = inputs["token_type_ids"].to(device)
    mask = inputs["attention_mask"].to(device)
    # forward pass
    outputs = model(ids,token_type_ids=idt,attention_mask=mask)
    logits = outputs[0]
    x = F.softmax(logits, dim=-1)
    active_logits = logits.view(-1, model.num_labels)  # shape (batch_size * seq_len, num_labels)
    flattened_predictions = torch.argmax(active_logits,
                                         axis=1)
    return x.cpu().detach().numpy()[0][1], flattened_predictions.cpu().numpy()[0]

In [ ]:
#upload test data, still tr.csv
uploaded = files.upload()

Prepare test data

In [ ]:
import pandas as pd
df = pd.read_csv("tr.csv")

In [ ]:
sentences = df.irsen_text.values.tolist()[7000:]
real = df.claim_s.values.tolist()[7000:]
len(sentences)

Predict the test set

In [ ]:
pre = []# predict label
pre_pro = []# predict probility
for i in sentences:
  x = predict(i)
  pre.append(x[1])
  pre_pro.append(x[0])

Show result

In [ ]:
print('f1:'+str(f1_score(real, pre, average=None))+'\n'+'recall:'+str(recall_score(real, pre, average=None))+'\n'+'precision:'+str(precision_score(real, pre, average=None))+'\n'+'accuracy:'+str(accuracy_score(real, pre))+'\n')

In [ ]:
confusion_matrix(real, pre)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
fig, ax = plt.subplots(1,1,figsize = (10,5))
ax.hist(np.array(pre_pro)[np.array(real) == 1], color = "darkred",bins = "scott", alpha = .5, edgecolor = "red")
ax.hist(np.array(pre_pro)[np.array(real) == 0], color = "darkgreen",bins = "scott", alpha = .5, edgecolor = "green")

In [ ]:
# number of samples model gives probabilty more than .8 but real label are non-evidence
print('num of samples have score more than 0.8 but are non-evidence : '+ str(np.sum(np.array(pre_pro)[np.array(real) == 0]>=0.8)))
print('num of samples have score more than 0.8 are non-evidence : '+ str(np.sum(np.array(pre_pro)[np.array(real) == 1]>=0.8)))
# how much confidence if the socore is higher than 0.8, we are 85.7% confident that the sentence is evidence if the score is higher than .8
confi80 = str(round((np.sum(np.array(pre_pro)[np.array(real) == 1]>=0.8))/(np.sum(np.array(pre_pro)[np.array(real) == 1]>=0.8) + np.sum(np.array(pre_pro)[np.array(real) == 0]>=0.8)),4))
print('we are '+ confi80 +' confident that the sentence is evidence if the score is higher than .8')
# number of samples model gives probabilty more than .6 less than .8 but real label is non-evidence
num6080_nevid = np.sum(np.array(pre_pro)[np.array(real) == 0]>=0.6) - np.sum(np.array(pre_pro)[np.array(real) == 0]>=0.80)
num6080_evid = np.sum(np.array(pre_pro)[np.array(real) == 1]>=0.6) - np.sum(np.array(pre_pro)[np.array(real) == 1]>=0.80)
print('num of samples have score more than 0.6 and less than 0.8 but are non-evidence :'+ str(num6080_nevid))
print('num of samples have score more than 0.6 and less than 0.8 are evidence :'+ str(num6080_evid))
confi6080 = num6080_evid/(num6080_evid + num6080_nevid)
print('we are '+ str(round(confi6080,4)) +' confident that the sentence is evidence if the score is higher than .6 and less than .8')

In [ ]:
print('num of samples have score more than 0.8 but are non-evidence : '+ str(np.sum(np.array(pre_pro)[np.array(real) == 0]<=0.2)))
print('num of samples have score more than 0.8 are non-evidence : '+ str(np.sum(np.array(pre_pro)[np.array(real) == 1]<=0.2)))
confi20 = str(round((np.sum(np.array(pre_pro)[np.array(real) == 0]<=0.2))/(np.sum(np.array(pre_pro)[np.array(real) == 0]<=0.2) + np.sum(np.array(pre_pro)[np.array(real) == 1]<=0.2)),4))
print('we are '+ confi80 +' confident that the sentence is non-evidence if the score is higher than .2')
num2040_nevid = np.sum(np.array(pre_pro)[np.array(real) == 0]<=0.4) - np.sum(np.array(pre_pro)[np.array(real) == 0]<=0.2)
num2040_evid = np.sum(np.array(pre_pro)[np.array(real) == 1]<=0.4) - np.sum(np.array(pre_pro)[np.array(real) == 1]<=0.2)
print('num of samples have score more than 0.2 and less than 0.4 but are non-evidence :'+ str(num2040_nevid))
print('num of samples have score more than 0.2 and less than 0.4 are evidence :'+ str(num2040_evid))
confi2040 = num2040_nevid/(num2040_evid + num2040_nevid)
print('we are '+ str(round(confi2040,4)) +' confident that the sentence is non-evidence if the score is higher than .2 and less than .4')

In [ ]:
# print the wrong predictions. The end of sentence shows the real labels
for i in range(len(real)):
  if real[i] != pre[i]:
    print(sentences[i] + ' claim' if real[i] else sentences[i] + ' noclaim')